Import the relevant packages

In [1]:
import numpy as np
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/CS224N

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/CS224N


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(re.sub(r'[^a-zA-Z\s]', '', stop_word) for stop_word in stopwords.words('english'))
print(stop_words)

{'a', 'wouldn', 'shouldve', 'her', 'having', 'isnt', 'havent', 'on', 'not', 'youre', 'his', 'shouldnt', 'up', 'himself', 'under', 'wouldnt', 'didnt', 'dont', 'if', 'and', 'werent', 'them', 'against', 'has', 'before', 'ma', 'thatll', 'after', 'as', 'few', 'will', 'd', 'yourself', 'during', 'should', 'shan', 'myself', 'below', 'by', 'some', 'll', 'does', 'of', 'down', 'is', 'youve', 'mightnt', 'its', 'o', 'couldn', 'through', 'once', 'than', 'into', 'him', 'same', 'while', 'both', 'so', 'again', 's', 'hadn', 'ours', 'because', 'being', 'about', 'in', 'which', 'doing', 'did', 'out', 'can', 'above', 'then', 'with', 'am', 'our', 'yours', 'now', 'your', 'itself', 'he', 'further', 'mustn', 'ourselves', 'i', 'isn', 'theirs', 'do', 'youd', 'haven', 'shouldn', 'shant', 'from', 'but', 'that', 'm', 'weren', 'be', 'no', 'at', 'between', 'such', 'themselves', 'hasn', 'don', 're', 'ain', 'aren', 'neednt', 'for', 'mustnt', 'doesnt', 'she', 'mightn', 'herself', 'how', 'any', 'doesn', 'over', 'yourselve

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
seed_val = 42

load in data

In [5]:
!ls
train_df = pd.read_csv('train_model_input.csv')
test_df = pd.read_csv('test_model_input.csv')
val_df = pd.read_csv('val_model_input.csv')

'Ana - Diabetes VB NoWeights bert_medium.ipynb'
'Ana - Hypertension VB NoWeights bert_medium.ipynb'
'Ana transformers_multi-label_classification.ipynb'
'AWS Copy of Ana - Diabetes VB NoWeights bert_medium.ipynb'
'AWS  Copy of Ana - Hypertension VB NoWeights bert_medium.ipynb'
 bert-finetuned-sem_eval-english
 bert_medium.ipynb
 bert_med_unweighted.pt
 bert_weights.pt
 bow_classifier.ipynb
 Bow_Classifier.ipynb
 clean_model_input.csv
 Clinical_Bert.ipynb
 Clinical_Bert_Single_Label.ipynb
'Copy of 11.multi-label-text-classification-with-bert.ipynb'
'Copy of Ana - Diabetes VB NoWeights bert_medium.ipynb'
'Copy of BertMultiLabelClassifier.ipynb'
'Copy of Data_Processing_Milestone.ipynb'
'Copy of Om Copy of bert_medium.ipynb'
'Copy of Om Copy of bow_classifier.ipynb'
'Copy of Om - Hypertension CB NoWeights bert_medium.ipynb'
'Copy of Peyton - Diabetes CB NoWeights bert_medium.ipynb'
'Copy of Peyton - Diabetes CB Weights bert_medium.ipynb'
'Copy of Peyton'\''s Fine-tuning BERT (and friends) 

Preprocess Data

save preprocessed data

make it more readable as input for our model

In [6]:
train_df['text'] = train_df.apply(lambda row: ' '.join(row['text'].split()[:318]), axis=1)

In [7]:
test_df['text'] = test_df.apply(lambda row: ' '.join(row['text'].split()[:318]), axis=1)

In [8]:
x_train, y_train = train_df['text'].values.tolist(), train_df['hypertension'].values.tolist()

In [9]:
len(x_train[0].split())

318

In [10]:
x_test, y_test = test_df['text'].values.tolist(), test_df['hypertension'].values.tolist()

In [11]:
del train_df, test_df, val_df

Create training and validation split

In [12]:
train_Y = np.array(list(y_train)).T[:5000]  # is a 2d numpy array, with a row for each comorbidity
valid_Y = np.array(list(y_test)).T[:500]   # ^^^
print(train_Y.shape)
print(valid_Y.shape)

(5000,)
(500,)


In [13]:
count_vectorizer = CountVectorizer()
train_X = count_vectorizer.fit_transform(x_train[:5000]).toarray()
print(train_X.shape)
valid_X = count_vectorizer.transform(x_test[:500]).toarray()
print(valid_X.shape)

(5000, 34348)
(500, 34348)


In [14]:
del x_train, x_test

Make predictions and print out f1 score based for each comorbidity

In [15]:
comorbidities = ['hypertension']
train_f1s, valid_f1s = None, None
train_pred_Y, valid_pred_Y = None, None

for i in range(len(comorbidities)):     # iterates across each comorbidity
    comorbidity, train_y, valid_y = comorbidities[i], train_Y, valid_Y
    count_clf = LogisticRegression(max_iter=10000).fit(train_X, train_y) # train the classifier

    train_pred_Y = count_clf.predict(train_X)
    train_f1s = f1_score(train_y, train_pred_Y)
    print(f"{comorbidity} training f1: {train_f1s}")

    valid_pred_Y = count_clf.predict(valid_X)
    valid_f1s = f1_score(valid_y, valid_pred_Y)
    print(f"{comorbidity} validation f1: {valid_f1s}")

flat_train_Y, flat_train_pred_Y = train_Y, train_pred_Y
micro_f1_1 = f1_score(flat_train_Y, flat_train_pred_Y,average='micro')
macro_f1_1 = f1_score(flat_train_Y, flat_train_pred_Y,average='macro')
tn, fp, fn, tp = confusion_matrix(flat_train_Y, flat_train_pred_Y).ravel()
print("\nTraining")
print(f"\tmicro f1: {micro_f1_1}")
print(f"\tmacro f1: {macro_f1_1}")
print(f"\ttn: {tn}\tfp: {fp}\tfn: {fn}\ttp: {tp}")

flat_valid_Y, flat_valid_prep_Y = valid_Y, valid_pred_Y
micro_f1 = f1_score(flat_valid_Y, flat_valid_prep_Y,average='micro')
macro_f1 = f1_score(flat_valid_Y, flat_valid_prep_Y,average='macro')
tn, fp, fn, tp = confusion_matrix(flat_valid_Y, flat_valid_prep_Y).ravel()
print("\nValidation")
print(f"\tmicro f1: {micro_f1}")
print(f"\tmacro f1: {macro_f1}")
print(f"\ttn: {tn}\tfp: {fp}\tfn: {fn}\ttp: {tp}")

hypertension training f1: 1.0
hypertension validation f1: 0.048

Training
	micro f1: 1.0
	macro f1: 1.0
	tn: 4795	fp: 0	fn: 0	tp: 205

Validation
	micro f1: 0.286
	macro f1: 0.2384
	tn: 134	fp: 2	fn: 355	tp: 9
